In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
code_departement = "093"
logs_level = "WARNING"

# Imports & setup

In [ ]:
import geopandas as gpd

from potentiel_solaire.constants import DATA_FOLDER
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)

In [ ]:
path_results = DATA_FOLDER / "093_pipeline_results.gpkg"

### Etablissements scolaires


In [ ]:
schools_establishments = gpd.read_file(path_results, layer="schools_establishments")
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

### Zones d'éducations

In [ ]:
educational_zones = gpd.read_file(path_results, layer="educational_zones")
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

# Détermination des bâtiments scolaires

In [ ]:
schools_buildings = gpd.read_file(path_results, layer="schools_buildings")

# Calcul des attributs utiles pour le potentiel solaire

In [ ]:
# TODO: v0 seulement à ce stade
solar_potential_of_schools_buildings = gpd.read_file(path_results, layer="solar_potential_of_schools_buildings")

# Check nos calculs vs potentiel solaire des toitures

In [ ]:
bool_Audit = True

In [ ]:
if code_departement == "093" and bool_Audit:
    # Audit des données sur Département 93
    GPKG = DATA_FOLDER / "potentiel-solaire.geojson"
    gspsdt_total = gpd.read_file(GPKG)
    print(len(gspsdt_total),"batiments")
    gspsdt_total=  gspsdt_total.to_crs(4326)

In [ ]:
if code_departement == "093" and bool_Audit:
    batiments_a_auditer = solar_potential_of_schools_buildings.cleabs_bat.unique()
    gspsdt = gspsdt_total[gspsdt_total.id.isin(batiments_a_auditer)]

    gspsdt = gspsdt[["id","st_areashape","surf_util","moyenne2","production"]]
    comparaison = gspsdt.merge(solar_potential_of_schools_buildings[["cleabs_bat","surface_totale_au_sol","surface_utile","rayonnement_solaire","potentiel_solaire"]],\
             left_on='id', right_on='cleabs_bat', how="inner")
    
    total_ecole         = comparaison.production.sum()
    total_ecole_calcule = comparaison.potentiel_solaire.sum()
    print("# Verification des potentiels pour:",code_departement,"\n")
    print("* Verification effectuée sur:",len(comparaison),"batiments.")
    print("* BDD Potentiel solaire\t",int(total_ecole),"kWh/an")
    print("* Estimation ordre 0\t",int(total_ecole_calcule),"kWh/an")
    print("* Tentative accuracy\t",int(total_ecole/total_ecole_calcule*100),"%")

#### Observations noms des batiments

In [ ]:
lack = len(batiments_a_auditer)-len(gspsdt)
print("* There are",len(batiments_a_auditer),"batiments in",code_departement)
print("* There are",len(gspsdt),"batiments in the solar potential database for these buildings")
print("* Coverage of that DB (%):",100*len(gspsdt)/len(batiments_a_auditer))

In [ ]:
dupes = solar_potential_of_schools_buildings[solar_potential_of_schools_buildings.duplicated(subset=["cleabs_bat"],keep=False)]
dupes = dupes.sort_values(by=["cleabs_bat"]).reset_index(drop=True)
print("* Duplicate rate for buildings in output (%):",100*len(dupes)/len(solar_potential_of_schools_buildings))
dupes[["cleabs_bat","cleabs_zone"]]

In [ ]:
len(solar_potential_of_schools_buildings),len(solar_potential_of_schools_buildings.cleabs_bat.unique())

In [ ]:
len(gspsdt_total[gspsdt_total.id.isin(batiments_a_auditer)])

# Checks sur la qualité des données & calculs

In [ ]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))